In [ ]:
import numpy as np
import pandas as pd
import cv2
from sklearn.cluster import KMeans
import pickle
from scipy.spatial.distance import cdist
import os
import glob


In [ ]:
import numpy as np
import pandas as pd
import os
import torchvision.datasets
from torch.utils.data import DataLoader, Subset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read dataset
trainLabels = pd.read_csv('/kaggle/input/supervised-sets/train_labels.csv')
testLabels = pd.read_csv('//kaggle/input/supervised-sets/val_labels.csv')

trainClasses = trainLabels['label'].unique()
testClasses = testLabels['label'].unique()

In [ ]:
train_dir = '/kaggle/input/supervised-sets/processedData/processedData/processed_train_set'
val_dir = '/kaggle/input/supervised-sets/processedData/processedData/processed_val_set' # test directory

In [ ]:
size = 224

fake_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((size, size))
    ])


# creation of the training set of resized images
fake_training_set = torchvision.datasets.ImageFolder(root='/kaggle/input/supervised-sets/processedData/processedData/processed_train_set', transform=fake_transforms)
# divide the images in batches: "fake" loader in order to compute mean and std for normalization
fake_train_loader = DataLoader(fake_training_set, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
# compute the mean and standard deviation for the normalization
def tot_mean_std(loader):
    mean = 0
    std = 0
    count = 0
    for batch, _ in loader:
        batch_samples = batch.size(0)
        batch = batch.view(batch_samples, -1)
        mean += batch.mean(1).sum(0)  # mean over the pixels of each image of the batch summed to the others
        std += batch.std(1).sum(0)  # same for the standard deviation
        count += batch_samples

    mean /= count
    std /= count

    return mean, std

In [ ]:
size = 224
mean, std = tot_mean_std(fake_train_loader)

transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((size, size)),
        torchvision.transforms.Normalize(mean = mean, std = std),
        torchvision.transforms.Grayscale()
    ])

training_set = torchvision.datasets.ImageFolder(root='/kaggle/input/supervised-sets/processedData/processedData/processed_train_set', transform=transforms)
test_set = torchvision.datasets.ImageFolder(root='/kaggle/input/supervised-sets/processedData/processedData/processed_test_set', transform=transforms)


# get 30% from any classes to reduce the dimension of the dataset
def get_subset(dataset, subset_percentage):
    class_indices = {}
    for idx, (_, label) in enumerate(dataset.samples):
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(idx)

    subset_indices = []
    for label, indices in class_indices.items():
        np.random.shuffle(indices)
        n_subset = int(len(indices) * subset_percentage)
        subset_indices.extend(indices[:n_subset])

    return Subset(dataset, subset_indices)

# get the subset
subset_dataset = get_subset(training_set, subset_percentage=0.3)



# normalized "true" dataloaders

train_loader = DataLoader(subset_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
print(mean)
print(std)

In [ ]:
# check if the images in the subset corresponds to their class
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# iter for only two batches
for i, data in enumerate(train_loader, 0):
    if i >= 2:  # Fermarsi dopo 2 batch
        break

    data_train, train_labels = data

    # print images and labels
    for j in range(len(data_train)):
        print(f"Etichetta: {training_set.classes[train_labels[j]]}")
        imshow(data_train[j])
        # stop
        break

In [ ]:
len(subset_dataset)

In [ ]:
len(training_set)

In [ ]:
# function to extract sift features
def sift_features(image_list):
    descriptors = []
    valid_indices = []
    sift = cv2.SIFT_create()
    for i, image in enumerate(image_list):
        if image is not None:
            image_np = image.numpy().transpose(1, 2, 0).astype(np.uint8)
            _, descriptor = sift.detectAndCompute(image_np, None)
            if descriptor is not None:
                descriptors.append(descriptor)
                valid_indices.append(i)
        else:
            print("Null image")

    return descriptors, valid_indices

In [ ]:
# create BOW dictionary with k-means applied on SIFT descriptors to compute centroids
# G is the number of words in the vocaboulary
def bow_dictionary(descriptors, G):
    bow_dict = []

    kmeans = KMeans(n_clusters = G)
    kmeans.fit(descriptors)

    bow_dict = kmeans.cluster_centers_

    if not os.path.isfile('bow_dictionary.pkl'):
        pickle.dump(bow_dict, open('bow_dictionary.pkl', 'wb'))

    return bow_dict

In [ ]:
# create the histogram of bow features
def bow_features(descriptors, centers, G):
    bow_features = []
    for descriptor_set in descriptors:
        for descriptor in descriptor_set:
            if descriptor is not None:
                features = np.zeros(G)
                distance = cdist(descriptor, centers)
                minimum = np.argmin(distance, axis=1)
                for index in minimum:
                    features[index] += 1
                bow_features.append(features)
            else:
                print("Null descriptor")
    return bow_features

In [ ]:
# extract descriptors
import sklearn
import argparse
from sklearn.model_selection import GridSearchCV

all_train_descriptors = []
training_descriptors = []
training_labels = []
valid_indices_all = []

for i, data in enumerate(train_loader, 0):
    data_train, train_labels = data
    train_descriptors, valid_indices = sift_features(data_train)
    training_descriptors.append(train_descriptors)
    training_labels.extend(train_labels[valid_indices])
    valid_indices_all.extend(valid_indices)
    for descriptor in train_descriptors:
        if descriptor is not None:
            all_train_descriptors.extend(descriptor)

In [ ]:
print(len(valid_indices_all))

In [ ]:
print(len(training_labels))

In [ ]:
# create dictionary, and bow_features

G = 380
bow_dict_train = bow_dictionary(all_train_descriptors, G)
train_features = bow_features(training_descriptors, bow_dict_train, G)


In [ ]:
svm_model = sklearn.svm.SVC(C = 30, random_state = 0)

In [ ]:
training_labels = [tensor.item() for tensor in training_labels]

In [ ]:
print(training_labels[:10])  # print the firste 10 values as example

In [ ]:
Y_train = training_labels

In [ ]:
# fit the model (SVM)
svm_model.fit(train_features, Y_train)
filename = 'svm_model.sav'
pickle.dump(svm_model, open(filename, 'wb'))

In [ ]:
print(len(Y_train))

In [ ]:
print(len(train_features))

In [ ]:
# repeat for the test set

all_test_descriptors = []
testing_descriptors = []
testing_labels = []
valid_indices_all_test = []

for i, data in enumerate(test_loader, 0):
    data_test, test_labels = data
    test_descriptors, valid_indices_test = sift_features(data_test)
    testing_descriptors.append(test_descriptors)
    testing_labels.extend(test_labels[valid_indices_test])
    valid_indices_all_test.extend(valid_indices_test)
    for test_descriptor in test_descriptors:
        if test_descriptor is not None:
            all_test_descriptors.extend(test_descriptor)



test_features = bow_features(testing_descriptors, bow_dict_train, G) # attention: the dictionary is the same of the training set
testing_labels = [tensor.item() for tensor in testing_labels]
Y_test = testing_labels
# do predictions using SVM
predictions = svm_model.predict(test_features)

# compute accuracy
accuracy = sklearn.metrics.accuracy_score(Y_test, predictions)
print(f"Accuracy on test set: {accuracy:.4f}")